In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from diffusers import StableDiffusionPipeline
from torch.optim import AdamW
from tqdm import tqdm
import numpy as np

In [ ]:
class StableDiffusionFinetuner:
    def __init__(self, 
                 model_name="stabilityai/stable-diffusion-2-1-base", 
                 epochs=10,  # Setting the number of epochs to 10
                 batch_size=4,
                 output_dir="/kaggle/working/fine_tuned_images"):
        """
        Initializing the fine-tuning process with configurable parameters.
        
        Args:
            model_name (str): The name of the Stable Diffusion model to use
            epochs (int): Number of training epochs
            batch_size (int): The batch size for training
            output_dir (str): Directory path to save generated images
        """
        # Setting the device for running the model, using GPU if available
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {self.device}")

        # Initializing model parameters based on user input
        self.model_name = model_name
        self.epochs = epochs
        self.batch_size = batch_size
        self.output_dir = output_dir

        # Creating the directory to save generated images if it doesn't exist
        os.makedirs(self.output_dir, exist_ok=True)

        # Loading the Stable Diffusion model into the device
        self.pipe = self.load_model()

        # Preparing the data loader for training data
        self.dataloader = self.prepare_data()

        # Setting up an optimizer (AdamW) for model fine-tuning
        self.optimizer = AdamW(self.pipe.unet.parameters(), lr=1e-5)

    def load_model(self):
        """Loading the Stable Diffusion model from the Hugging Face model hub."""
        try:
            # Loading the pre-trained model and moving it to the appropriate device (GPU or CPU)
            pipe = StableDiffusionPipeline.from_pretrained(self.model_name).to(self.device)
            return pipe
        except Exception as e:
            # Handling any errors during model loading
            print(f"Error loading model: {e}")
            raise

    def prepare_data(self):
        """Preparing the CIFAR-10 dataset with necessary transformations."""
        # Defining image transformation to resize, convert to tensor, and normalize
        transform = transforms.Compose([
            transforms.Resize((512, 512)),  # Resizing images to 512x512
            transforms.ToTensor(),  # Converting images to tensor format
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalizing for pre-trained models
        ])
        
        # Downloading and loading the CIFAR-10 dataset with the specified transformations
        cifar10 = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
        
        # Creating a DataLoader for batching, shuffling, and parallel data loading
        dataloader = DataLoader(
            cifar10, 
            batch_size=self.batch_size,  # Using the specified batch size
            shuffle=True,  # Shuffling the dataset for better training
            num_workers=2,  # Using 2 workers to load data in parallel
            pin_memory=True  # Pinning memory for faster data transfer to GPU
        )
        
        return dataloader

    def generate_images(self, epoch, batch_idx, img, label):
        """
        Generating images using two methods (text-to-image, image-to-image) with error handling.
        
        Args:
            epoch (int): The current training epoch
            batch_idx (int): The current batch index within the epoch
            img (torch.Tensor): The input image tensor from the dataloader
            label (torch.Tensor): The label of the image (for class name)
        """
        try:
            # Converting the image tensor to a PIL Image for generation
            img_pil = transforms.ToPILImage()(img.cpu())
            
            # Retrieving the class name for the label to use in the generation prompts
            class_name = self.dataloader.dataset.classes[label.item()]
            
            # Method 1: Generating an image from text prompt
            try:
                text_prompt = f"A high-quality, detailed image of a {class_name}"
                # Generating an image from the text prompt
                text_generated = self.pipe(
                    prompt=text_prompt, 
                    num_inference_steps=50,  # Number of inference steps for generation
                    guidance_scale=7.5  # Using higher guidance scale for better quality
                ).images[0]
                
                # Defining the path to save the generated image
                text_save_path = os.path.join(
                    self.output_dir, 
                    f"text_gen_epoch_{epoch+1}_batch_{batch_idx+1}.png"
                )
                text_generated.save(text_save_path)  # Saving the generated image
                print(f"Saved text-to-image: {text_save_path}")
            except Exception as text_gen_error:
                print(f"Text-to-image generation error: {text_gen_error}")
            
            # Method 2: Image-to-image generation using the input image as the base
            try:
                img2img_prompt = f"Enhanced, photorealistic version of a {class_name}"
                # Generating a new image based on the input image and text prompt
                img2img_generated = self.pipe(
                    prompt=img2img_prompt, 
                    init_image=img_pil,  # Initial image for the transformation
                    strength=0.75,  # Controlling how much of the initial image is preserved
                    num_inference_steps=50,  # Number of inference steps for generation
                    guidance_scale=7.5  # Using higher guidance scale for better quality
                ).images[0]
                
                # Defining the path to save the generated image
                img2img_save_path = os.path.join(
                    self.output_dir, 
                    f"img2img_gen_epoch_{epoch+1}_batch_{batch_idx+1}.png"
                )
                img2img_generated.save(img2img_save_path)  # Saving the generated image
                print(f"Saved image-to-image: {img2img_save_path}")
            except Exception as img2img_gen_error:
                print(f"Image-to-image generation error: {img2img_gen_error}")

        except Exception as overall_error:
            print(f"Overall image generation error: {overall_error}")

    def fine_tune(self):
        """Main fine-tuning method with comprehensive error handling."""
        print("Starting fine-tuning process...")
        
        # Iterating over the number of epochs
        for epoch in range(self.epochs):
            print(f"Epoch {epoch+1}/{self.epochs}")
            
            # Creating a progress bar for monitoring the training process
            progress_bar = tqdm(
                self.dataloader, 
                desc=f"Epoch {epoch+1}", 
                unit="batch"
            )
            
            # Iterating through batches in the dataloader
            for batch_idx, (img, label) in enumerate(progress_bar):
                try:
                    # Moving image and label tensors to the device (GPU or CPU)
                    img = img.to(self.device)
                    label = label.to(self.device)
                    
                    # Generating images for this batch using the generate_images method
                    self.generate_images(epoch, batch_idx, img[0], label[0])
                    
                    # Updating the progress bar with current batch details
                    progress_bar.set_postfix({
                        "Batch": batch_idx+1,  # Current batch number
                        "Label": self.dataloader.dataset.classes[label[0].item()]  # Current batch label (class)
                    })
                    
                except Exception as batch_error:
                    # Handling errors during batch processing
                    print(f"Error in batch {batch_idx}: {batch_error}")
                    continue
            
            # Indicating successful completion of the current epoch
            print(f"Epoch {epoch+1} completed successfully!")
        
        print("Fine-tuning process completed.")

In [ ]:
def main():
    # Initializing the fine-tuning process with specified parameters
    fine_tuner = StableDiffusionFinetuner(
        model_name="stabilityai/stable-diffusion-2-1-base",  # Using Stable Diffusion model
        epochs=10,  # Setting number of epochs to 10
        batch_size=4  # Using batch size of 4
    )
    # Running the fine-tuning process
    fine_tuner.fine_tune()

if __name__ == "__main__":
    main()
